In [1]:
import os, gc, torch, PIL, pickle,json,io

import pandas as pd
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from utils.plot import plot_losses
from PIL import Image
from models.load import get_model_label, get_model_name, get_current_epoch, get_dataset_label, get_trained_model

from data.constants import DEFAULT_REFLACX_LABEL_COLS

import utils.print as print_f
from utils.init import reproducibility, clean_memory_get_device

# from models.load import TrainedModels
## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class ImageSizeTest(Enum):
    size128 = "lesion-detection_val_ap_0_0856_test_ap_0_0693_epoch67_05-23-2023 13-53-13_baseline_mobilenet"
    size512 = "lesion-detection_val_ap_0_1283_test_ap_0_1174_epoch42_05-23-2023 16-38-06_baseline_mobilenet"

class APNatureModels(Enum):
    baseline_mobilenet = "lesion-detection_val_ap_0_1281_test_ap_0_1180_epoch49_05-17-2023 18-16-58_baseline_mobilenet"
    clinical_mobilenet_sum = "lesion-detection_val_ap_0_1542_test_ap_0_1294_epoch60_05-17-2023 21-23-17_clinical_mobilenet_sum"
    clinical_mobilenet_hadamard = "lesion-detection_val_ap_0_1296_test_ap_0_1427_epoch35_05-17-2023 23-33-46_clinical_mobilenet_hadamard"
    clinical_mobilenet_concat_conv = "lesion-detection_val_ap_0_1272_test_ap_0_1639_epoch35_05-18-2023 01-43-56_clinical_mobilenet_concat_conv"
    clinical_mobilenet_concat_linear = "lesion-detection_val_ap_0_1294_test_ap_0_1415_epoch55_05-18-2023 04-57-29_clinical_mobilenet_concat_linear"
    clinical_mobilenet_concat_convb = "lesion-detection_val_ap_0_1252_test_ap_0_1343_epoch61_05-18-2023 09-09-04_clinical_mobilenet_concat_convb"
    clinical_mobilenet_concat_conv_nb = "lesion-detection_val_ap_0_1220_test_ap_0_1149_epoch42_05-18-2023 22-56-01_clinical_mobilenet_concat_convb_nb"
    clinical_mobilenet_concat_conv_inter = "lesion-detection_val_ap_0_1233_test_ap_0_1402_epoch42_05-19-2023 01-13-45_clinical_mobilenet_concat_conv_inter"
    clinical_mobilenet_concat_conv_repeat = "lesion-detection_val_ap_0_1348_test_ap_0_1446_epoch60_05-19-2023 04-13-39_clinical_mobilenet_concat_conv_repeat"
    clinical_mobilenet_concat_deform = "lesion-detection_val_ap_0_1405_test_ap_0_1231_epoch47_05-19-2023 07-34-33_clinical_mobilenet_concat_deform"

class FROCNatureModels(Enum):

    baseline_mobilenet = "lesion-detection_val_froc_0_8257_test_froc_0_7691_epoch35_05-17-2023 17-46-23_baseline_mobilenet"
    clinical_mobilenet_sum = "lesion-detection_val_froc_0_8014_test_froc_0_7331_epoch31_05-17-2023 20-03-34_clinical_mobilenet_sum"
    clinical_mobilenet_hadamard = "lesion-detection_val_froc_0_8369_test_froc_0_7545_epoch25_05-17-2023 23-06-47_clinical_mobilenet_hadamard"
    clinical_mobilenet_concat_conv = "lesion-detection_val_froc_0_8185_test_froc_0_8069_epoch29_05-18-2023 01-26-37_clinical_mobilenet_concat_conv"
    clinical_mobilenet_concat_linear = "lesion-detection_val_froc_0_8233_test_froc_0_7907_epoch32_05-18-2023 03-48-49_clinical_mobilenet_concat_linear"
    clinical_mobilenet_concat_convb = "lesion-detection_val_froc_0_8047_test_froc_0_7226_epoch32_05-18-2023 07-51-15_clinical_mobilenet_concat_convb"
    clinical_mobilenet_concat_convb_nb = "lesion-detection_val_froc_0_8020_test_froc_0_7185_epoch30_05-18-2023 22-28-31_clinical_mobilenet_concat_convb_nb"
    clinical_mobilenet_concat_conv_inter = "lesion-detection_val_froc_0_8402_test_froc_0_7701_epoch26_05-19-2023 00-31-44_clinical_mobilenet_concat_conv_inter"
    clinical_mobilenet_concat_conv_repeat = "lesion-detection_val_froc_0_8312_test_froc_0_7636_epoch27_05-19-2023 02-54-07_clinical_mobilenet_concat_conv_repeat"
    clinical_mobilenet_concat_deform = "lesion-detection_val_froc_0_7905_test_froc_0_7824_epoch25_05-19-2023 06-14-09_clinical_mobilenet_concat_deform"

class NatureModels(Enum):
    clinical_convnext_base_sum_ap_3D = "lesion-detection_val_ap_0_1375_test_ap_0_1823_epoch22_05-16-2023 18-23-36_clinical_convnext_base_sum"
    clinical_convnext_base_sum_froc_3D = "lesion-detection_val_froc_0_8378_test_froc_0_7804_epoch26_05-16-2023 18-36-07_clinical_convnext_base_sum"
    clinical_convnext_base_sum_ar_3D = "lesion-detection_val_ar_0_6835_test_ar_0_6754_epoch39_05-16-2023 19-17-06_clinical_convnext_base_sum"
    baseline_convnext_base_ap = 'lesion-detection_val_ap_0_1560_test_ap_0_1567_epoch52_05-16-2023 22-20-39_baseline_convnext_base'
    baseline_convnext_base_froc = 'lesion-detection_val_froc_0_8560_test_froc_0_8079_epoch59_05-16-2023 22-47-45_baseline_convnext_base'
    baseline_convnext_base_ar = 'lesion-detection_val_ar_0_7054_test_ar_0_7032_epoch60_05-16-2023 22-51-45_baseline_convnext_base'
    clinical_convnext_base_sum_ap_1D_3D = "lesion-detection_val_ap_0_1373_test_ap_0_1393_epoch34_05-17-2023 02-45-30_clinical_convnext_base_sum"
    clinical_convnext_base_sum_froc_1D_3D = "lesion-detection_val_froc_0_8189_test_froc_0_7287_epoch157_05-17-2023 12-15-01_clinical_convnext_base_sum"
    clinical_convnext_base_sum_ar_1D_3D = "lesion-detection_val_ar_0_7309_test_ar_0_6901_epoch60_05-17-2023 07-00-34_clinical_convnext_base_sum"

class FROCEarlyStopModels(Enum):
    baseline_mobilenet = "val_lesion-detection_froc_0_8214_test_lesion-detection_froc_0_7215_epoch29_05-08-2023 03-29-15_lesion_dsetection_baseline_mobilenet"
    with_clinical_mobilenet_1D_3D = "val_lesion-detection_froc_0_7802_test_lesion-detection_froc_0_7193_epoch27_05-08-2023 04-57-59_lesion_dsetection_with_clinical_mobilenet"
    with_clinical_mobilenet_3D = "val_lesion-detection_froc_0_8345_test_lesion-detection_froc_0_7381_epoch31_05-08-2023 08-28-01_lesion_dsetection_with_clinical_mobilenet"
    baseline_resnet18 = "val_lesion-detection_froc_0_8262_test_lesion-detection_froc_0_7828_epoch21_05-08-2023 10-52-48_lesion_dsetection_baseline_resnet18"
    with_clinical_resnet18_1D_3D = "val_lesion-detection_froc_0_8063_test_lesion-detection_froc_0_7156_epoch21_05-08-2023 17-24-35_lesion_dsetection_with_clinical_resnet18"
    with_clinical_resnet18_3D = "val_lesion-detection_froc_0_8391_test_lesion-detection_froc_0_7007_epoch51_05-09-2023 00-03-18_lesion_dsetection_with_clinical_resnet18"
    baseline_convnext_base = "val_lesion-detection_froc_0_8422_test_lesion-detection_froc_0_7991_epoch39_05-09-2023 03-20-36_lesion_dsetection_baseline_convnext_base"
    with_clinical_convnext_base_1D_3D = "val_lesion-detection_froc_0_8248_test_lesion-detection_froc_0_7867_epoch45_05-09-2023 07-31-28_lesion_dsetection_with_clinical_convnext_base_1D_3D"
    with_clinical_convnext_base_3D = "val_lesion-detection_froc_0_8104_test_lesion-detection_froc_0_7584_epoch62_05-09-2023 12-56-34_lesion_dsetection_with_clinical_convnext_base_3D"
    with_clinical_convnext_base_3D_2nd = "val_lesion-detection_froc_0_8148_test_lesion-detection_froc_0_7649_epoch39_05-09-2023 17-27-41_lesion_dsetection_with_clinical_convnext_base_3D"
    with_clinical_convnext_base_1D_3D_2nd = "val_lesion-detection_froc_0_8409_test_lesion-detection_froc_0_8003_epoch29_05-09-2023 19-09-54_lesion_dsetection_with_clinical_convnext_base_1D_3D"
    baseline_convnext_base_2nd = "val_lesion-detection_froc_0_8597_test_lesion-detection_froc_0_8035_epoch41_05-09-2023 21-40-22_lesion_dsetection_baseline_convnext_base"
    baseline_efficientnet_b0 = "val_lesion-detection_froc_0_8301_test_lesion-detection_froc_0_7479_epoch23_05-09-2023 23-23-11_lesion_dsetection_baseline_efficientnet_b0"
    with_clinical_efficientnet_b0_1D_3D = "val_lesion-detection_froc_0_7876_test_lesion-detection_froc_0_7051_epoch35_05-10-2023 07-17-37_lesion_dsetection_with_clinical_efficientnet_b0_1D_3D"
    with_clinical_efficientnet_b0_3D = "val_lesion-detection_froc_0_8321_test_lesion-detection_froc_0_7246_epoch25_05-10-2023 09-19-11_lesion_dsetection_with_clinical_efficientnet_b0_3D"
    baseline_densenet = "val_lesion-detection_froc_0_8442_test_lesion-detection_froc_0_6942_epoch33_05-10-2023 17-06-03_lesion_dsetection_baseline_densenet"
    with_clinical_densenet_1D_3D = "val_lesion-detection_froc_0_8303_test_lesion-detection_froc_0_7089_epoch33_05-10-2023 19-11-45_lesion_dsetection_with_clinical_densenet_1D_3D"
    with_clinical_densenet_3D  = "val_lesion-detection_froc_0_8310_test_lesion-detection_froc_0_7144_epoch12_05-10-2023 20-13-22_lesion_dsetection_with_clinical_densenet_3D"
    baseline_resnet50 = "val_lesion-detection_froc_0_8350_test_lesion-detection_froc_0_7959_epoch15_05-10-2023 21-30-40_lesion_dsetection_baseline_resnet50"
    with_clinical_resnet50_1D_3D = "val_lesion-detection_froc_0_8119_test_lesion-detection_froc_0_7458_epoch15_05-10-2023 22-32-10_lesion_dsetection_with_clinical_resnet50_1D_3D"
    with_clinical_resnet50_3D = "val_lesion-detection_froc_0_8260_test_lesion-detection_froc_0_7630_epoch15_05-10-2023 23-35-14_lesion_dsetection_with_clinical_resnet50_3D"
    baseline_efficientnet_b5 = "val_lesion-detection_froc_0_8016_test_lesion-detection_froc_0_7337_epoch35_05-11-2023 01-44-17_lesion_dsetection_baseline_efficientnet_b5"
    with_clinical_efficientnet_b5_1D_3D = "val_lesion-detection_froc_0_8027_test_lesion-detection_froc_0_6531_epoch26_05-11-2023 03-42-43_lesion_dsetection_with_clinical_efficientnet_b5_1D_3D"
    with_clinical_efficientnet_b5_3D = "val_lesion-detection_froc_0_8209_test_lesion-detection_froc_0_6131_epoch34_05-11-2023 06-05-03_lesion_dsetection_with_clinical_efficientnet_b5_3D"
    with_clinical_convnext_base_3D_hadamard = "val_lesion-detection_froc_0_8236_test_lesion-detection_froc_0_7954_epoch42_05-11-2023 14-58-09_lesion_dsetection_with_clinical_convnext_base_3D_hadamard"

    # changed file names.
    with_clinical_convnext_base_3D_cocat_token_mixer = "val_froc_0_7650_test_froc_0_7231_epoch25_05-11-2023 19-32-51_clinical_convnext_3D_cocat_token_mixer"
    with_clinical_convnext_base_3D_hadamard_2nd = "val_froc_0_8186_test_froc_0_7648_epoch61_05-11-2023 22-43-35_clinical_convnext_3D_hadamard"
    with_clinical_expander_convnext_base_3D_cocat = "val_froc_0_8453_test_froc_0_7186_epoch51_05-12-2023 02-57-57_clinical_expander_convnext_3D_cocat"
    with_clinical_convnext_base_3D_cocat = "val_froc_0_7863_test_froc_0_7991_epoch13_05-12-2023 04-03-12_clinical_convnext_3D_cocat"
    with_clinical_convnext_base_3D_cocat_with_norm_act_token_mixer = "val_froc_0_7976_test_froc_0_7680_epoch28_05-12-2023 15-26-58_clinical_convnext_3D_cocat_with_mixer"
    with_clinical_convnext_base_3D_cocat_pos_weight_10 = "val_froc_0_8070_test_froc_0_7986_epoch18_05-12-2023 21-53-37_clinical_convnext_3D_cocat_pos_weight_10"
    with_clinical_convnext_base_3D_cocat_pos_weight_100 = "val_froc_0_7888_test_froc_0_7747_epoch13_05-12-2023 23-17-06_clinical_convnext_3D_cocat_pos_weight_100"
    with_clinical_convnext_base_3D_cocat_pos_weight_10_2nd = "val_froc_0_8227_test_froc_0_7324_epoch70_05-13-2023 04-26-25_clinical_convnext_3D_cocat_pos_weight_10"
    with_clinical_convnext_base_3D_cocat_pos_weight_100_2nd  = "val_froc_0_8216_test_froc_0_7314_epoch65_05-13-2023 07-51-16_clinical_convnext_3D_cocat_pos_weight_100"

    ap_lesion_dsetection_with_clinical_convnext_base_3D = "val_lesion-detection_ap_0_1290_test_lesion-detection_ap_0_1564_epoch60_05-14-2023 00-32-39_lesion_dsetection_with_clinical_convnext_base_3D"
    ap_lesion_dsetection_with_clinical_convnext_base_1D_3D = "val_lesion-detection_ap_0_0970_test_lesion-detection_ap_0_1072_epoch56_05-14-2023 03-54-58_lesion_dsetection_with_clinical_convnext_base_1D_3D"
    ap_lesion_dsetection_baseline_convnext_base = "val_lesion-detection_ap_0_1309_test_lesion-detection_ap_0_1245_epoch56_05-14-2023 07-09-35_lesion_dsetection_baseline_convnext_base"
    ap_clinical_efficientnet_b0_3D =  "val_lesion-detection_ap_0_1138_test_lesion-detection_ap_0_1088_epoch68_05-14-2023 17-09-14_clinical_efficientnet_b0_3D"
    ap_clinical_efficientnet_b0_1D_3D =  "val_lesion-detection_ap_0_1202_test_lesion-detection_ap_0_1081_epoch30_05-14-2023 19-10-56_clinical_efficientnet_b0_1D_3D"
    ap_baseline_efficientnet_b0 =  "val_lesion-detection_ap_0_1127_test_lesion-detection_ap_0_1156_epoch36_05-14-2023 21-39-07_baseline_efficientnet_b0"
    
    ap_clinical_densenet_3D = "val_lesion-detection_ap_0_1240_test_lesion-detection_ap_0_1129_epoch56_05-15-2023 13-04-42_clinical_densenet_3D"
    ap_clinical_densenet_1D_3D = "val_lesion-detection_ap_0_1182_test_lesion-detection_ap_0_1200_epoch26_05-15-2023 16-25-55_clinical_densenet_1D_3D"
    ap_baseline_densenet = "val_lesion-detection_ap_0_1382_test_lesion-detection_ap_0_1230_epoch21_05-15-2023 18-19-01_baseline_densenet"

    # file names too long.
    # with_clinical_convnext_base_3D_cocat_token_mixer = "val_lesion-detection_froc_0_7650_test_lesion-detection_froc_0_7231_epoch25_05-11-2023 19-32-51_lesion_dsetection_with_clinical_convnext_base_3D_cocat_token_mixer"
    # with_clinical_convnext_base_3D_hadamard_2nd = "val_lesion-detection_froc_0_8186_test_lesion-detection_froc_0_7648_epoch61_05-11-2023 22-43-35_lesion_dsetection_with_clinical_convnext_base_3D_hadamard"
    # with_clinical_expander_convnext_base_3D_cocat = "val_lesion-detection_froc_0_8453_test_lesion-detection_froc_0_7186_epoch51_05-12-2023 02-57-57_lesion_dsetection_with_clinical_expander_convnext_base_3D_cocat"
    # with_clinical_convnext_base_3D_cocat = "val_lesion-detection_froc_0_7863_test_lesion-detection_froc_0_7991_epoch13_05-12-2023 04-03-12_lesion_dsetection_with_clinical_convnext_base_3D_cocat"
    # with_clinical_convnext_base_3D_cocat_with_norm_act_token_mixer = "val_lesion-detection_froc_0_7976_test_lesion-detection_froc_0_7680_epoch28_05-12-2023 15-26-58_lesion_dsetection_with_clinical_convnext_base_3D_cocat_with_norm_act_token_mixer"
    # with_clinical_convnext_base_3D_cocat_pos_weight_10 = "val_lesion-detection_froc_0_8070_test_lesion-detection_froc_0_7986_epoch18_05-12-2023 21-53-37_lesion_dsetection_with_clinical_convnext_base_3D_cocat_pos_weight_10"
    # with_clinical_convnext_base_3D_cocat_pos_weight_100 = "val_lesion-detection_froc_0_7888_test_lesion-detection_froc_0_7747_epoch13_05-12-2023 23-17-06_lesion_dsetection_with_clinical_convnext_base_3D_cocat_pos_weight_100"
    # with_clinical_convnext_base_3D_cocat_pos_weight_10_2nd = "val_lesion-detection_froc_0_8227_test_lesion-detection_froc_0_7324_epoch70_05-13-2023 04-26-25_lesion_dsetection_with_clinical_convnext_base_3D_cocat_pos_weight_10"
    # with_clinical_convnext_base_3D_cocat_pos_weight_100_2nd  = "val_lesion-detection_froc_0_8216_test_lesion-detection_froc_0_7314_epoch65_05-13-2023 07-51-16_lesion_dsetection_with_clinical_convnext_base_3D_cocat_pos_weight_100"

class RetrainedModels(Enum):
    mobilenet_with_fix = "val_lesion-detection_froc_0_7916_test_lesion-detection_froc_0_8150_epoch15_04-01-2023 03-02-43_lesion_dsetection_with_fix_mobilenet"
    mobilenet_baseline = "val_lesion-detection_froc_0_7764_test_lesion-detection_froc_0_8036_epoch8_04-01-2023 07-46-01_lesion_dsetection_baseline_mobilenet"
    resnet18_with_fix = "val_lesion-detection_froc_0_7982_test_lesion-detection_froc_0_8257_epoch5_04-01-2023 11-24-08_lesion_dsetection_with_fix_resnet18"
    resnet18_baseline = "val_lesion-detection_froc_0_7893_test_lesion-detection_froc_0_8075_epoch5_04-01-2023 13-53-42_lesion_dsetection_baseline_resnet18"
    densenet_with_fix = "val_lesion-detection_froc_0_8004_test_lesion-detection_froc_0_8169_epoch12_04-02-2023 03-08-03_lesion_dsetection_with_fix_densenet"
    densenet_baseline = "val_lesion-detection_froc_0_7841_test_lesion-detection_froc_0_7864_epoch7_04-02-2023 06-38-49_lesion_dsetection_baseline_densenet"
    efficientnet_b5_with_fix = "val_lesion-detection_froc_0_7980_test_lesion-detection_froc_0_8006_epoch7_04-02-2023 19-25-56_lesion_dsetection_with_fix_efficientnet_b5"
    efficientnet_b5_baseline = "val_lesion-detection_froc_0_7846_test_lesion-detection_froc_0_8189_epoch5_04-02-2023 22-14-38_lesion_dsetection_baseline_efficientnet_b5"
    efficientnet_b0_with_fix = "val_lesion-detection_froc_0_8030_test_lesion-detection_froc_0_8089_epoch7_04-02-2023 12-42-11_lesion_dsetection_with_fix_efficientnet_b0"
    efficientnet_b0_baseline = "val_lesion-detection_froc_0_7939_test_lesion-detection_froc_0_8115_epoch5_04-02-2023 15-17-34_lesion_dsetection_baseline_efficientnet_b0"
    vgg16_with_fix = "val_lesion-detection_froc_0_8004_test_lesion-detection_froc_0_8247_epoch9_04-03-2023 01-53-49_lesion_dsetection_with_fix_vgg16"
    vgg16_baseline = "val_lesion-detection_froc_0_7903_test_lesion-detection_froc_0_7991_epoch8_04-03-2023 04-59-07_lesion_dsetection_baseline_vgg16"
    regnet_y_8gf_with_fix ="val_lesion-detection_froc_0_8059_test_lesion-detection_froc_0_8284_epoch7_04-03-2023 10-01-30_lesion_dsetection_with_fix_regnet_y_8gf"
    regnet_y_8gf_baseline = "val_lesion-detection_froc_0_7862_test_lesion-detection_froc_0_8172_epoch5_04-03-2023 14-04-49_lesion_dsetection_baseline_regnet_y_8gf"
    convnext_base_with_fix = "val_lesion-detection_froc_0_8141_test_lesion-detection_froc_0_8278_epoch7_04-03-2023 17-07-24_lesion_dsetection_with_fix_convnext_base"
    convnext_base_baseline = "val_lesion-detection_froc_0_7903_test_lesion-detection_froc_0_8092_epoch11_04-03-2023 21-17-10_lesion_dsetection_baseline_convnext_base"
    resnet50_with_fix = "val_lesion-detection_froc_0_8201_test_lesion-detection_froc_0_8307_epoch7_04-04-2023 01-13-30_lesion_dsetection_with_fix_resnet50"
    resnet50_baseline = "val_lesion-detection_froc_0_7882_test_lesion-detection_froc_0_8111_epoch6_04-04-2023 04-00-17_lesion_dsetection_baseline_resnet50"
    resnet50_with_fix_silent = "val_lesion-detection_froc_0_7947_test_lesion-detection_froc_0_8172_epoch8_04-04-2023 22-46-16_ld_with_fix_resnet50_silent"
    resnet50_with_fix_full = "val_lesion-detection_froc_0_8135_test_lesion-detection_froc_0_8361_epoch10_04-05-2023 04-40-29_ld_with_fix_resnet50_full"
    
class TrainedModels(Enum):
    mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2472_test_lesion-detection_ap_0_2637_epoch15_03-23-2023 20-20-07_lesion_dsetection_with_fixation_convnext_base_silent_report"
    # convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    convnext_base_with_fix_silent = "val_lesion-detection_ap_0_2405_test_lesion-detection_ap_0_2543_epoch19_03-24-2023 13-57-29_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix_full = "val_lesion-detection_ap_0_2602_test_lesion-detection_ap_0_2499_epoch22_03-24-2023 04-42-21_lesion_dsetection_with_fixation_convnext_base_full_report"
    vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [4]:
naming_map = {
    ImageSizeTest.size512: "size512",
    ImageSizeTest.size128: "size128",
    APNatureModels.baseline_mobilenet: "baseline_mobilenet",
    APNatureModels.clinical_mobilenet_sum: "clinical_mobilenet_sum",
    APNatureModels.clinical_mobilenet_hadamard: "clinical_mobilenet_hadamard",
    APNatureModels.clinical_mobilenet_concat_conv: "clinical_mobilenet_concat_conv",
    APNatureModels.clinical_mobilenet_concat_linear: "clinical_mobilenet_concat_linear",
    APNatureModels.clinical_mobilenet_concat_convb: "clinical_mobilenet_concat_convb",
    APNatureModels.clinical_mobilenet_concat_conv_nb: "clinical_mobilenet_concat_conv_nb",
    APNatureModels.clinical_mobilenet_concat_conv_inter: "clinical_mobilenet_concat_conv_inter",
    APNatureModels.clinical_mobilenet_concat_conv_repeat: "clinical_mobilenet_concat_conv_repeat",
    APNatureModels.clinical_mobilenet_concat_deform : "clinical_mobilenet_concat_deform",
    FROCNatureModels.baseline_mobilenet: "baseline_mobilenet",
    FROCNatureModels.clinical_mobilenet_sum: "clinical_mobilenet_sum",
    FROCNatureModels.clinical_mobilenet_hadamard: "clinical_mobilenet_hadamard",
    FROCNatureModels.clinical_mobilenet_concat_conv: "clinical_mobilenet_concat_conv",
    FROCNatureModels.clinical_mobilenet_concat_linear: "clinical_mobilenet_concat_linear",
    FROCNatureModels.clinical_mobilenet_concat_convb: "clinical_mobilenet_concat_convb",
    FROCNatureModels.clinical_mobilenet_concat_convb_nb: "clinical_mobilenet_concat_convb_nb",
    FROCNatureModels.clinical_mobilenet_concat_conv_inter: "clinical_mobilenet_concat_conv_inter",
    FROCNatureModels.clinical_mobilenet_concat_conv_repeat: "clinical_mobilenet_concat_conv_repeat",
    FROCNatureModels.clinical_mobilenet_concat_deform: "clinical_mobilenet_concat_deform",
    NatureModels.clinical_convnext_base_sum_ap_3D: "clinical_convnext_base_sum_ap_3D",
    NatureModels.clinical_convnext_base_sum_froc_3D: "clinical_convnext_base_sum_froc_3D",
    NatureModels.clinical_convnext_base_sum_ar_3D: "clinical_convnext_base_sum_ar_3D",
    NatureModels.baseline_convnext_base_ap: "baseline_convnext_base_ap",
    NatureModels.baseline_convnext_base_froc: "baseline_convnext_base_froc",
    NatureModels.baseline_convnext_base_ar: "baseline_convnext_base_ar",
    NatureModels.clinical_convnext_base_sum_ap_1D_3D: "clinical_convnext_base_sum_ap_1D_3D",
    NatureModels.clinical_convnext_base_sum_froc_1D_3D: "clinical_convnext_base_sum_froc_1D_3D",
    NatureModels.clinical_convnext_base_sum_ar_1D_3D: "clinical_convnext_base_sum_ar_1D_3D",
    FROCEarlyStopModels.baseline_mobilenet: "baseline_mobilenet",
    FROCEarlyStopModels.baseline_mobilenet: "baseline_mobilenet",
    FROCEarlyStopModels.with_clinical_mobilenet_1D_3D: "with_clinical_mobilenet_1D_3D",
    FROCEarlyStopModels.with_clinical_mobilenet_3D: "with_clinical_mobilenet_3D",
    FROCEarlyStopModels.baseline_resnet18: "baseline_resnet18",
    FROCEarlyStopModels.with_clinical_resnet18_1D_3D: "with_clinical_resnet18_1D_3D",
    FROCEarlyStopModels.with_clinical_resnet18_3D: "with_clinical_resnet18_3D",
    FROCEarlyStopModels.baseline_convnext_base: "baseline_convnext_base",
    FROCEarlyStopModels.with_clinical_convnext_base_1D_3D: "with_clinical_convnext_base_1D_3D",
    FROCEarlyStopModels.with_clinical_convnext_base_3D: "with_clinical_convnext_base_3D",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_2nd: "with_clinical_convnext_base_3D_2nd",
    FROCEarlyStopModels.with_clinical_convnext_base_1D_3D_2nd: "with_clinical_convnext_base_1D_3D_2nd",
    FROCEarlyStopModels.baseline_convnext_base_2nd: "baseline_convnext_base_2nd",
    FROCEarlyStopModels.baseline_efficientnet_b0: "baseline_efficientnet_b0",
    FROCEarlyStopModels.with_clinical_efficientnet_b0_1D_3D: "with_clinical_efficientnet_b0_1D_3D",
    FROCEarlyStopModels.with_clinical_efficientnet_b0_3D: "with_clinical_efficientnet_b0_3D",
    FROCEarlyStopModels.baseline_densenet: "baseline_densenet",
    FROCEarlyStopModels.with_clinical_densenet_1D_3D: "with_clinical_densenet_1D_3D",
    FROCEarlyStopModels.with_clinical_densenet_3D: "with_clinical_densenet_3D",
    FROCEarlyStopModels.baseline_resnet50: "baseline_resnet50",
    FROCEarlyStopModels.with_clinical_resnet50_1D_3D: "with_clinical_resnet50_1D_3D",
    FROCEarlyStopModels.with_clinical_resnet50_3D: "with_clinical_resnet50_3D",
    FROCEarlyStopModels.baseline_efficientnet_b5: "baseline_efficientnet_b5",
    FROCEarlyStopModels.with_clinical_efficientnet_b5_1D_3D: "with_clinical_efficientnet_b5_1D_3D",
    FROCEarlyStopModels.with_clinical_efficientnet_b5_3D: "with_clinical_efficientnet_b5_3D",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_hadamard: "with_clinical_convnext_base_3D_hadamard",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_token_mixer: "with_clinical_convnext_base_3D_cocat_token_mixer",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_hadamard_2nd: "with_clinical_convnext_base_3D_hadamard_2nd",
    FROCEarlyStopModels.with_clinical_expander_convnext_base_3D_cocat: "with_clinical_expander_convnext_base_3D_cocat",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat: "with_clinical_convnext_base_3D_cocat",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_with_norm_act_token_mixer: "with_clinical_convnext_base_3D_cocat_with_norm_act_token_mixer",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_pos_weight_10: "with_clinical_convnext_base_3D_cocat_pos_weight_10",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_pos_weight_100: "with_clinical_convnext_base_3D_cocat_pos_weight_100",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_pos_weight_10_2nd: "with_clinical_convnext_base_3D_cocat_pos_weight_10_2nd",
    FROCEarlyStopModels.with_clinical_convnext_base_3D_cocat_pos_weight_100_2nd: "with_clinical_convnext_base_3D_cocat_pos_weight_100_2nd",
    FROCEarlyStopModels.ap_lesion_dsetection_with_clinical_convnext_base_3D: "ap_lesion_dsetection_with_clinical_convnext_base_3D",
    FROCEarlyStopModels.ap_lesion_dsetection_with_clinical_convnext_base_1D_3D: "ap_lesion_dsetection_with_clinical_convnext_base_1D_3D",
    FROCEarlyStopModels.ap_lesion_dsetection_baseline_convnext_base: "ap_lesion_dsetection_baseline_convnext_base",
    FROCEarlyStopModels.ap_clinical_efficientnet_b0_3D: "ap_clinical_efficientnet_b0_3D",
    FROCEarlyStopModels.ap_clinical_efficientnet_b0_1D_3D: "ap_clinical_efficientnet_b0_1D_3D",
    FROCEarlyStopModels.ap_baseline_efficientnet_b0: "ap_baseline_efficientnet_b0",
    FROCEarlyStopModels.ap_clinical_densenet_3D: "ap_clinical_densenet_3D",
    FROCEarlyStopModels.ap_clinical_densenet_1D_3D: "ap_clinical_densenet_1D_3D",
    FROCEarlyStopModels.ap_baseline_densenet: "ap_baseline_densenet",
    TrainedModels.mobilenet_baseline: "mobilenet_baseline (ap)",
    TrainedModels.mobilenet_with_fix: "mobilenet_with_fix (ap)",
    TrainedModels.resnet18_baseline: "resnet18_baseline (ap)",
    TrainedModels.resnet18_with_fix: "resnet18_with_fix (ap)",
    TrainedModels.densenet161_baseline: "densenet161_baseline (ap)",
    TrainedModels.densenet161_with_fix: "densenet161_with_fix (ap)",
    TrainedModels.efficientnet_b5_baseline: "efficientnet_b5_baseline (ap)",
    TrainedModels.efficientnet_b5_with_fix: "efficientnet_b5_with_fix (ap)",
    TrainedModels.efficientnet_b0_baseline: "efficientnet_b0_baseline (ap)",
    TrainedModels.efficientnet_b0_with_fix: "efficientnet_b0_with_fix (ap)",
    TrainedModels.convnext_base_with_fix: "convnext_base_with_fix (ap)",
    TrainedModels.convnext_base_baseline: "convnext_base_baseline (ap)",
    TrainedModels.convnext_base_with_fix_silent: "convnext_base_with_fix_silent (ap)",
    TrainedModels.convnext_base_with_fix_full: "convnext_base_with_fix_full (ap)",
    TrainedModels.vgg16_with_fix: "vgg16_with_fix (ap)",
    TrainedModels.vgg16_baseline: "vgg16_baseline (ap)",
    TrainedModels.regnet_y_8gf_with_fix: "regnet_y_8gf_with_fix (ap)",
    TrainedModels.regnet_y_8gf_baseline: "regnet_y_8gf_baseline (ap)",
    RetrainedModels.mobilenet_with_fix: "mobilenet_with_fix (froc)",
    RetrainedModels.mobilenet_baseline: "mobilenet_baseline (froc)",
    RetrainedModels.resnet18_with_fix: "resnet18_with_fix (froc)",
    RetrainedModels.resnet18_baseline: "resnet18_baseline (froc)",
    RetrainedModels.densenet_with_fix: "densenet_with_fix (froc)",
    RetrainedModels.densenet_baseline: "densenet_baseline (froc)",
    RetrainedModels.efficientnet_b5_with_fix: "efficientnet_b5_with_fix (froc)",
    RetrainedModels.efficientnet_b5_baseline: "efficientnet_b5_baseline (froc)",
    RetrainedModels.efficientnet_b0_with_fix: "efficientnet_b0_with_fix (froc)",
    RetrainedModels.efficientnet_b0_baseline: "efficientnet_b0_baseline (froc)",
    RetrainedModels.vgg16_with_fix: "vgg16_with_fix (froc)",
    RetrainedModels.vgg16_baseline: "vgg16_baseline (froc)",
    RetrainedModels.regnet_y_8gf_with_fix: "regnet_y_8gf_with_fix (froc)",
    RetrainedModels.regnet_y_8gf_baseline: "regnet_y_8gf_baseline (froc)",
    RetrainedModels.convnext_base_with_fix: "convnext_base_with_fix (froc)",
    RetrainedModels.convnext_base_baseline: "convnext_base_baseline (froc)",
    RetrainedModels.resnet50_with_fix: "resnet50_with_fix (froc)",
    RetrainedModels.resnet50_baseline: "resnet50_baseline (froc)",
    RetrainedModels.resnet50_with_fix_silent: "resnet50_with_fix (silent) (froc)",
    RetrainedModels.resnet50_with_fix_full: "resnet50_with_fix (full) (froc)",
}


In [5]:
score_thrs="0.05"

In [6]:
from pathlib import Path

In [7]:
eval_ss = []

save_prefix = "testing_trained"
for select_model in TrainedModels:
    model_path = select_model.value
    for l in DEFAULT_REFLACX_LABEL_COLS + ["all"]:
        eval_df = pd.read_csv(
            Path(os.path.join(
                "./eval_results",
                f"{model_path}_{l}_{score_thrs}.csv",
            )),
            index_col=0,
        )
        eval_s = eval_df.iloc[0]
        eval_s["model"] = naming_map[select_model]
        eval_s["lesion"] = l
        eval_s["score_threshold"] = score_thrs
        states = json.loads(eval_s["coco_states"])
        for s in states:
            eval_s[s["p_title_str"]] = s["mean_s"]
        eval_ss.append(eval_s)


In [8]:
eval_df = pd.DataFrame(eval_ss)

In [9]:
eval_df

,num_fps,num_fns,num_tps,coco_states,Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",model,...,Average Precision (AP) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ]
0,938,84,115,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.608543,0.738693,0.812864,0.888883,0.762246,mobilenet_baseline (ap),...,-1,-1,0.099477,0.149749,0.386432,0.437186,0.454271,-1,-1,0.437186
0,448,134,143,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.608303,0.740452,0.864081,0.957607,0.792611,mobilenet_baseline (ap),...,-1,-1,0.172026,0.248375,0.408303,0.414440,0.414440,-1,-1,0.414440
0,1092,139,198,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.507418,0.747774,0.879130,0.937494,0.767954,mobilenet_baseline (ap),...,-1,-1,0.114860,0.168546,0.356973,0.404451,0.408309,-1,-1,0.404451
0,643,182,135,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.322668,0.500631,0.713767,0.906087,0.610788,mobilenet_baseline (ap),...,-1,-1,0.051029,0.150158,0.277287,0.291483,0.294006,-1,-1,0.291483
0,269,306,98,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.462120,0.635768,0.800206,0.889389,0.696871,mobilenet_baseline (ap),...,-1,-1,0.044417,0.087871,0.146040,0.149010,0.149010,-1,-1,0.149010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1357,79,198,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.491233,0.657538,0.840353,0.951384,0.735127,regnet_y_8gf_baseline (ap),...,-1,-1,0.171514,0.289170,0.512635,0.570036,0.573646,-1,-1,0.570036
0,2154,107,230,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.462908,0.652350,0.809918,0.951344,0.719130,regnet_y_8gf_baseline (ap),...,-1,-1,0.093429,0.167953,0.429970,0.484570,0.510089,-1,-1,0.484570
0,1073,132,185,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.308623,0.579090,0.760197,0.877924,0.631458,regnet_y_8gf_baseline (ap),...,-1,-1,0.058112,0.171609,0.367508,0.384227,0.384227,-1,-1,0.384227
0,988,227,177,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.381403,0.555747,0.715246,0.861552,0.628487,regnet_y_8gf_baseline (ap),...,-1,-1,0.052436,0.121040,0.256188,0.275743,0.278713,-1,-1,0.275743


In [10]:
# fig, ax = plt.subplots(figsize=[25, 13])

# ins = ax.inset_axes([0.55, 0.05, 0.45, 0.4])
# ins.set_xticks(
#     [0.1, 1.0, 2.0, 3.0, 4.0],
#     [
#         0.1,
#         1.0,
#         2.0,
#         3.0,
#         4.0,
#     ],
#     fontsize=30,
# )

# ins.set_xlim([0.1, 4.5])

# for idx, eval_instance in eval_df[eval_df["lesion"] == "all"].iterrows():
#     froc_curve = json.loads(eval_instance["froc_curve"])
#     x_axis = json.loads(eval_instance["x-axis"])
#     name = eval_instance["model"]
#     ax.semilogx(
#         x_axis,
#         froc_curve,
#         "x--",
#         label=name,
#     )
#     ins.plot(
#         x_axis,
#         froc_curve,
#         "x--",
#         label=name,
#     )


# box = ax.get_position()
# ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# ax.legend(
#     loc="center left",
#     # bbox_to_anchor=(0.8, 0.6),
#     bbox_to_anchor=(1, 0.5),

#     fancybox=True,
#     shadow=True,
#     ncol=1,
#     fontsize=25,
# )

# ax.set_title("FROC", fontdict={"fontsize": 35})
# ax.set_ylabel("Sensitivity", fontdict={"fontsize": 30})
# ax.set_xlabel("FP / image", fontdict={"fontsize": 30})

# ax.tick_params(axis="both", which="major", labelsize=30)
# ins.tick_params(axis="both", which="major", labelsize=20)

# ax.set_ylim(bottom=0.05, top=1.02)
# fig.tight_layout(pad=4.0)
# fig.savefig(fname=f"{save_prefix}_models_froc.png", dpi=150)

In [11]:
def get_early_stop_strategy_from_name(name: str):
    return "Average Precision" if name.startswith("ap_") else "FROC"

def get_fusion_method_from_name(name: str):
    if "1D" in name and "3D" in name:
        return "1D+3D"
    elif "1D" in name:
        return "1D"
    elif "3D" in name:
        return "3D"
    else:
        return ""

# eval_df['Early Stop'] = eval_df['model'].apply(get_early_stop_strategy_from_name)
# eval_df['Fusion'] = eval_df['model'].apply(get_fusion_method_from_name)

In [12]:
eval_df.columns

Index(['num_fps', 'num_fns', 'num_tps', 'coco_states',
       'Sensitivity@ [avgFP=0.5]', 'Sensitivity@ [avgFP=1]',
       'Sensitivity@ [avgFP=2]', 'Sensitivity@ [avgFP=4]', 'mFROC@[0.5,1,2,4]',
       'model', 'lesion', 'score_threshold',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]',
       ' Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]',
       ' Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 10 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ]',
       ' Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ]',
       ' Average Reca

In [13]:
# " Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ]" in eval_df.columns
# ' Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]'

In [14]:
# eval_df = eval_df[[
#     "model",
#     "lesion",
#     # "Early Stop",
#     # "Fusion",
#     "num_tps",
#     "num_fps",
#     "num_fns",
#     "Sensitivity@ [avgFP=0.5]",
#     "Sensitivity@ [avgFP=1]",
#     "Sensitivity@ [avgFP=2]",
#     "Sensitivity@ [avgFP=4]",
#     "mFROC@[0.5,1,2,4]",
#     " Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ]",
#     " Average Recall     (AR) @[ IoBB=0.50      | area=   all | maxDets= 10 ]",
#     " Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets= 30 ]",
#     " Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=  1 ]",
#     " Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets=  5 ]",
#     " Average Precision  (AP) @[ IoBB=0.50:0.95 | area=   all | maxDets= 10 ]",
#     " Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 30 ]",
#     " Average Precision  (AP) @[ IoBB=0.75      | area=   all | maxDets= 10 ]",
#     " Average Recall     (AR) @[ IoBB=0.50      | area=   all | maxDets= 30 ]",
#     " Average Recall     (AR) @[ IoBB=0.50:0.95 | area=   all | maxDets=  1 ]",
#     " Average Recall     (AR) @[ IoBB=0.50:0.95 | area=   all | maxDets=  5 ]",
#     " Average Recall     (AR) @[ IoBB=0.50:0.95 | area=   all | maxDets= 10 ]",
#     " Average Recall     (AR) @[ IoBB=0.50:0.95 | area=   all | maxDets= 30 ]",
# ]]

eval_df = eval_df[[
    "model",
    "lesion",
    # "Early Stop",
    # "Fusion",
    "num_tps",
    "num_fps",
    "num_fns",
    "Sensitivity@ [avgFP=0.5]",
    "Sensitivity@ [avgFP=1]",
    "Sensitivity@ [avgFP=2]",
    "Sensitivity@ [avgFP=4]",
    "mFROC@[0.5,1,2,4]",
    " Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]",
    # " Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    # " Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 10 ]",
    # " Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 30 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
]]



In [15]:
eval_df = eval_df.rename(columns={"num_tps": "#TP @[ IoBB=0.50      | area=   all | maxDets= 30 ]", "num_fps": "#FP @[ IoBB=0.50      | area=   all | maxDets= 30 ]", "num_fns":"#FN @[ IoBB=0.50      | area=   all | maxDets= 30 ]"})

In [16]:
eval_df

,model,lesion,#TP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FN @[ IoBB=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ]
0,mobilenet_baseline (ap),Pulmonary edema,115,938,84,0.608543,0.738693,0.812864,0.888883,0.762246,0.162356,0.100805,0.053652,0.094508,0.099462,0.086241,0.149749,0.386432,0.437186,0.454271
0,mobilenet_baseline (ap),Enlarged cardiac silhouette,143,448,134,0.608303,0.740452,0.864081,0.957607,0.792611,0.236619,0.171808,0.134515,0.171435,0.171808,0.181183,0.248375,0.408303,0.414440,0.414440
0,mobilenet_baseline (ap),Consolidation,198,1092,139,0.507418,0.747774,0.879130,0.937494,0.767954,0.228495,0.114949,0.070709,0.109587,0.114832,0.093292,0.168546,0.356973,0.404451,0.408309
0,mobilenet_baseline (ap),Atelectasis,135,643,182,0.322668,0.500631,0.713767,0.906087,0.610788,0.109484,0.051384,0.034454,0.049859,0.050992,0.034449,0.150158,0.277287,0.291483,0.294006
0,mobilenet_baseline (ap),Pleural abnormality,98,269,306,0.462120,0.635768,0.800206,0.889389,0.696871,0.099201,0.044183,0.034317,0.044903,0.044214,0.034263,0.087871,0.146040,0.149010,0.149010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,regnet_y_8gf_baseline (ap),Enlarged cardiac silhouette,198,1357,79,0.491233,0.657538,0.840353,0.951384,0.735127,0.234437,0.171091,0.131742,0.166215,0.171134,0.173545,0.289170,0.512635,0.570036,0.573646
0,regnet_y_8gf_baseline (ap),Consolidation,230,2154,107,0.462908,0.652350,0.809918,0.951344,0.719130,0.195110,0.095578,0.050339,0.090569,0.093340,0.073895,0.167953,0.429970,0.484570,0.510089
0,regnet_y_8gf_baseline (ap),Atelectasis,185,1073,132,0.308623,0.579090,0.760197,0.877924,0.631458,0.119215,0.057707,0.031901,0.056393,0.057708,0.046860,0.171609,0.367508,0.384227,0.384227
0,regnet_y_8gf_baseline (ap),Pleural abnormality,177,988,227,0.381403,0.555747,0.715246,0.861552,0.628487,0.117501,0.052194,0.041420,0.051205,0.052085,0.034475,0.121040,0.256188,0.275743,0.278713


In [17]:
eval_df.to_csv(f"{save_prefix}__eval.csv")

In [18]:
eval_df = eval_df[eval_df['lesion'] == 'all'] 
eval_df.to_csv(f"{save_prefix}__model_eval.csv")

In [19]:
eval_df

,model,lesion,#TP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FN @[ IoBB=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ]
0,mobilenet_baseline (ap),all,690,3387,844,0.488645,0.664704,0.812858,0.915661,0.720467,0.166417,0.096041,0.064508,0.093054,0.095719,0.084206,0.157808,0.313493,0.339928,0.345081
0,mobilenet_with_fix (ap),all,931,6485,603,0.520547,0.685702,0.829871,0.945574,0.745424,0.191661,0.110759,0.069133,0.104958,0.109648,0.102032,0.171908,0.382278,0.432620,0.447516
0,resnet18_baseline (ap),all,685,3014,849,0.547293,0.699487,0.844288,0.927435,0.754626,0.199014,0.109892,0.071348,0.106937,0.109438,0.096610,0.156954,0.317182,0.343033,0.347816
0,resnet18_with_fix (ap),all,878,5197,656,0.571132,0.734524,0.881214,0.969328,0.789049,0.222142,0.123580,0.070077,0.117872,0.122695,0.110252,0.171902,0.390436,0.438190,0.447970
0,densenet161_baseline (ap),all,849,4920,685,0.597284,0.796250,0.921627,0.966754,0.820479,0.210625,0.104538,0.060573,0.097924,0.103692,0.088013,0.149629,0.332777,0.385189,0.395612
0,densenet161_with_fix (ap),all,821,3983,713,0.563561,0.720557,0.860418,0.957039,0.775394,0.211696,0.122505,0.078322,0.116912,0.121789,0.109743,0.172558,0.364303,0.404433,0.414288
0,efficientnet_b5_baseline (ap),all,915,8657,619,0.565098,0.744209,0.886124,0.951194,0.786656,0.207596,0.126130,0.070580,0.116840,0.123789,0.115191,0.163638,0.391250,0.459441,0.491486
0,efficientnet_b5_with_fix (ap),all,819,3710,715,0.566572,0.751588,0.883266,0.960751,0.790544,0.220080,0.117471,0.072073,0.113261,0.116932,0.111972,0.168229,0.354105,0.384768,0.391246
0,efficientnet_b0_baseline (ap),all,691,2707,843,0.575332,0.745011,0.876680,0.946530,0.785888,0.186919,0.102695,0.067466,0.099586,0.102302,0.094496,0.157851,0.298919,0.320328,0.324508
0,efficientnet_b0_with_fix (ap),all,842,3755,692,0.580923,0.763835,0.894797,0.971172,0.802682,0.214829,0.123009,0.076214,0.118237,0.122478,0.115364,0.184708,0.374034,0.408440,0.414264


In [20]:
eval_df = eval_df[eval_df['lesion'] == 'all'].sort_values('mFROC@[0.5,1,2,4]',ascending=False)
eval_df.to_csv(f"{save_prefix}__eval_sort_froc.csv")

In [21]:
eval_df

,model,lesion,#TP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FN @[ IoBB=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ]
0,densenet161_baseline (ap),all,849,4920,685,0.597284,0.796250,0.921627,0.966754,0.820479,0.210625,0.104538,0.060573,0.097924,0.103692,0.088013,0.149629,0.332777,0.385189,0.395612
0,convnext_base_with_fix_silent (ap),all,821,3392,713,0.597851,0.782254,0.917028,0.975557,0.818172,0.249353,0.150139,0.095074,0.144533,0.149965,0.140911,0.187155,0.386224,0.425488,0.427377
0,vgg16_with_fix (ap),all,888,4228,646,0.582176,0.784087,0.913663,0.987333,0.816815,0.220570,0.120113,0.069960,0.115104,0.119823,0.111336,0.172412,0.389172,0.427477,0.430155
0,efficientnet_b0_with_fix (ap),all,842,3755,692,0.580923,0.763835,0.894797,0.971172,0.802682,0.214829,0.123009,0.076214,0.118237,0.122478,0.115364,0.184708,0.374034,0.408440,0.414264
0,vgg16_baseline (ap),all,918,6122,616,0.569839,0.753455,0.897395,0.969999,0.797672,0.211818,0.122350,0.074767,0.116769,0.121941,0.112443,0.177676,0.388790,0.437601,0.443413
0,convnext_base_baseline (ap),all,905,5145,629,0.573535,0.748165,0.901751,0.965384,0.797209,0.233165,0.138026,0.082400,0.130923,0.137361,0.131115,0.175081,0.391597,0.446859,0.457378
0,convnext_base_with_fix (ap),all,1032,6434,502,0.566653,0.747835,0.892043,0.975462,0.795498,0.265575,0.163421,0.094839,0.154157,0.161639,0.157577,0.196207,0.455144,0.521546,0.540668
0,efficientnet_b5_with_fix (ap),all,819,3710,715,0.566572,0.751588,0.883266,0.960751,0.790544,0.220080,0.117471,0.072073,0.113261,0.116932,0.111972,0.168229,0.354105,0.384768,0.391246
0,convnext_base_with_fix_full (ap),all,1010,6063,524,0.558193,0.730237,0.898569,0.969965,0.789241,0.249791,0.149078,0.080937,0.138865,0.147849,0.143288,0.182403,0.442326,0.510332,0.525052
0,resnet18_with_fix (ap),all,878,5197,656,0.571132,0.734524,0.881214,0.969328,0.789049,0.222142,0.123580,0.070077,0.117872,0.122695,0.110252,0.171902,0.390436,0.438190,0.447970


In [22]:
eval_df  = eval_df.sort_values(' Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]',ascending=False)
# eval_df  = eval_df.sort_values(' Average Precision  (AP) @[ IoBB=0.50      | area=   all | maxDets= 10 ]',ascending=False)
eval_df

,model,lesion,#TP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FP @[ IoBB=0.50 | area= all | maxDets= 30 ],#FN @[ IoBB=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ]
0,convnext_base_with_fix (ap),all,1032,6434,502,0.566653,0.747835,0.892043,0.975462,0.795498,0.265575,0.163421,0.094839,0.154157,0.161639,0.157577,0.196207,0.455144,0.521546,0.540668
0,convnext_base_with_fix_full (ap),all,1010,6063,524,0.558193,0.730237,0.898569,0.969965,0.789241,0.249791,0.149078,0.080937,0.138865,0.147849,0.143288,0.182403,0.442326,0.510332,0.525052
0,convnext_base_with_fix_silent (ap),all,821,3392,713,0.597851,0.782254,0.917028,0.975557,0.818172,0.249353,0.150139,0.095074,0.144533,0.149965,0.140911,0.187155,0.386224,0.425488,0.427377
0,convnext_base_baseline (ap),all,905,5145,629,0.573535,0.748165,0.901751,0.965384,0.797209,0.233165,0.138026,0.082400,0.130923,0.137361,0.131115,0.175081,0.391597,0.446859,0.457378
0,resnet18_with_fix (ap),all,878,5197,656,0.571132,0.734524,0.881214,0.969328,0.789049,0.222142,0.123580,0.070077,0.117872,0.122695,0.110252,0.171902,0.390436,0.438190,0.447970
0,vgg16_with_fix (ap),all,888,4228,646,0.582176,0.784087,0.913663,0.987333,0.816815,0.220570,0.120113,0.069960,0.115104,0.119823,0.111336,0.172412,0.389172,0.427477,0.430155
0,efficientnet_b5_with_fix (ap),all,819,3710,715,0.566572,0.751588,0.883266,0.960751,0.790544,0.220080,0.117471,0.072073,0.113261,0.116932,0.111972,0.168229,0.354105,0.384768,0.391246
0,efficientnet_b0_with_fix (ap),all,842,3755,692,0.580923,0.763835,0.894797,0.971172,0.802682,0.214829,0.123009,0.076214,0.118237,0.122478,0.115364,0.184708,0.374034,0.408440,0.414264
0,vgg16_baseline (ap),all,918,6122,616,0.569839,0.753455,0.897395,0.969999,0.797672,0.211818,0.122350,0.074767,0.116769,0.121941,0.112443,0.177676,0.388790,0.437601,0.443413
0,densenet161_with_fix (ap),all,821,3983,713,0.563561,0.720557,0.860418,0.957039,0.775394,0.211696,0.122505,0.078322,0.116912,0.121789,0.109743,0.172558,0.364303,0.404433,0.414288


In [23]:
eval_df.to_csv(f"{save_prefix}_model_eval_sort_ap.csv")